In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# import raw data and preview
airportRaw = pd.read_csv(r"C:\Users\britt\Documents\Python\airports\Airports.csv")
print(airportRaw.head())

   Rank                                            Airport  \
0     1       China Guangzhou Baiyun International Airport   
1     2  United States Hartsfield–Jackson Atlanta Inter...   
2     3      China Chengdu Shuangliu International Airport   
3     4  United States Dallas/Fort Worth International ...   
4     5        China Shenzhen Bao'an International Airport   

                             Location        Country      Code  Passengers  \
0  Baiyun-Huadu, Guangzhou, Guangdong          China  CAN/ZGGG    43760427   
1                    Atlanta, Georgia  United States  ATL/KATL    42918685   
2   Shuangliu-Wuhou, Chengdu, Sichuan          China  CTU/ZUUU    40741509   
3            Dallas-Fort Worth, Texas  United States  DFW/KDFW    39364990   
4         Bao'an, Shenzhen, Guangdong          China  SZX/ZGSZ    37916059   

   Year  
0  2020  
1  2020  
2  2020  
3  2020  
4  2020  


In [3]:
# check for missing values
count_nan = airportRaw.isna().sum().sum()
print('Count of missing values: ' + str(count_nan))

Count of missing values: 0


In [22]:
# strip leading and trailing white spaces
airportRaw = airportRaw.columns.str.replace(' ', '')

In [24]:
type(airportRaw)

pandas.core.indexes.base.Index

In [ ]:
# map with bubble chart in a particular size

In [20]:
airport.head()

AttributeError: 'Index' object has no attribute 'head'